In [2]:
import os
import math
import random
import argparse
import datetime
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import Xception
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard, CSVLogger
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import joblib
import pandas as pd
import logging

# ---------------- config ----------------
DATA_DIR = "./DataTrain/images"
VOLUME_FILES_DIR = "./DataTrain/labels"
MODEL_SAVE_PATH = "./mangosteen_volume_model_aug_finetune.h5"
CHECKPOINT_DIR = "./checkpoints"
SCALER_PATH = "./volume_scaler_finetune.save"
IMG_SIZE = 224
BATCH_SIZE = 32
INITIAL_EPOCHS = 100
FINE_TUNE_EPOCHS = 100
LEARNING_RATE_INITIAL = 1e-3
LEARNING_RATE_FINE_TUNE = 1e-8
RANDOM_SEED = 42
ENABLE_MIXED_PRECISION = False  # เปลี่ยนเป็น True ถ้าต้องการใช้ mixed precision (GPU + ระวัง output dtype)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# ---------------- reproducibility ----------------
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# ---------------- GPU safe config ----------------
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for g in gpus:
            tf.config.experimental.set_memory_growth(g, True)
        print("GPU found. Enabled memory growth.")
    except Exception as e:
        print("Could not set GPU memory growth:", e)

if ENABLE_MIXED_PRECISION:
    from tensorflow.keras import mixed_precision
    mixed_precision.set_global_policy('mixed_float16')
    print("Mixed precision enabled.")

# ---------------- logging ----------------
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s: %(message)s')

# --- augmentation helpers ---
def random_brightness_contrast(image):
    # image assumed float32 in [0,255]
    alpha = np.random.uniform(0.85, 1.15)
    beta = np.random.uniform(-0.1, 0.1) * 255.0
    out = image * alpha + beta
    return np.clip(out, 0.0, 255.0)

def safe_center_crop_resize(img, target_size):
    # ensure image at least target size by padding if necessary, then center-crop
    h, w = img.shape[:2]
    if h < target_size or w < target_size:
        pad_h = max(0, target_size - h)
        pad_w = max(0, target_size - w)
        top = pad_h // 2
        bottom = pad_h - top
        left = pad_w // 2
        right = pad_w - left
        img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_REFLECT)
        h, w = img.shape[:2]
    startx = (w - target_size) // 2
    starty = (h - target_size) // 2
    cropped = img[starty:starty+target_size, startx:startx+target_size]
    return cv2.resize(cropped, (target_size, target_size), interpolation=cv2.INTER_LINEAR)

def random_flip_rotate_scale_crop(img):
    # expects img float32 in [0,255]
    if random.random() < 0.5:
        img = cv2.flip(img, 1)
    # rotate
    angle = random.uniform(-15, 15)
    M = cv2.getRotationMatrix2D((img.shape[1]//2, img.shape[0]//2), angle, 1.0)
    img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]), borderMode=cv2.BORDER_REFLECT)
    # scale
    scale = random.uniform(0.9, 1.1)
    new_w = max(1, int(img.shape[1] * scale))
    new_h = max(1, int(img.shape[0] * scale))
    img = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
    img = safe_center_crop_resize(img, IMG_SIZE)
    return img

# --- Sequence implementation ---
class MangosteenSequence(Sequence):
    def __init__(self, image_paths, volumes, batch_size, img_size, is_training=True, shuffle=True):
        self.image_paths = list(image_paths)
        self.volumes = list(volumes)
        self.batch_size = batch_size
        self.img_size = img_size
        self.is_training = is_training
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.image_paths) / self.batch_size)

    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = min(start + self.batch_size, len(self.image_paths))
        batch_paths = self.image_paths[start:end]
        batch_vols = self.volumes[start:end]

        images = np.zeros((len(batch_paths), self.img_size, self.img_size, 3), dtype=np.float32)
        for i, p in enumerate(batch_paths):
            img = cv2.imread(p)
            if img is None:
                logging.warning(f"cv2.imread failed for {p}. Using black image.")
                img = np.zeros((self.img_size, self.img_size, 3), dtype=np.uint8)
            else:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (self.img_size, self.img_size), interpolation=cv2.INTER_LINEAR)
            img = img.astype('float32')
            if self.is_training:
                if random.random() < 0.9:
                    img = random_brightness_contrast(img)
                if random.random() < 0.7:
                    img = random_flip_rotate_scale_crop(img)
                # small gaussian noise occasionally
                if random.random() < 0.2:
                    noise = np.random.normal(0, 2.0, img.shape).astype(np.float32)
                    img = np.clip(img + noise, 0, 255)
            images[i] = img / 255.0
        vols = np.array(batch_vols, dtype='float32')
        return images, vols

    def on_epoch_end(self):
        if self.shuffle and self.is_training and len(self.image_paths) > 1:
            combined = list(zip(self.image_paths, self.volumes))
            random.shuffle(combined)
            self.image_paths, self.volumes = zip(*combined)
            self.image_paths = list(self.image_paths)
            self.volumes = list(self.volumes)

# --- load data ---
def load_data_from_folders(data_dir, volume_dir):
    image_paths = []
    volumes = []
    missing = 0
    bad_files = []
    for filename in os.listdir(data_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            base = os.path.splitext(filename)[0]
            vol_file = os.path.join(volume_dir, base + ".txt")
            if os.path.exists(vol_file):
                try:
                    with open(vol_file, 'r', encoding='utf-8') as f:
                        text = f.read().strip()
                        v = float(text)
                    image_paths.append(os.path.join(data_dir, filename))
                    volumes.append(v)
                except Exception as e:
                    bad_files.append((vol_file, str(e)))
            else:
                missing += 1
    if missing:
        logging.info(f"{missing} images found without matching .txt label files.")
    if bad_files:
        logging.warning(f"{len(bad_files)} label files failed to parse. Examples: {bad_files[:3]}")
    return image_paths, volumes

# ---------------- main pipeline ----------------
if __name__ == "__main__":
    print("Loading training data...")
    image_paths, volumes = load_data_from_folders(DATA_DIR, VOLUME_FILES_DIR)
    if not image_paths:
        raise SystemExit("❌ No data found. ตรวจสอบ DATA_DIR และ VOLUME_FILES_DIR")

    # split first to avoid leakage
    train_paths, val_paths, train_vols, val_vols = train_test_split(
        image_paths, volumes, test_size=0.2, random_state=RANDOM_SEED
    )

    # fit scaler on train only
    scaler = StandardScaler()
    train_vols_arr = np.array(train_vols).reshape(-1, 1)
    val_vols_arr = np.array(val_vols).reshape(-1, 1)
    scaler.fit(train_vols_arr)
    train_vols_scaled = scaler.transform(train_vols_arr).flatten()
    val_vols_scaled = scaler.transform(val_vols_arr).flatten()
    joblib.dump(scaler, SCALER_PATH)
    logging.info(f"Scaler saved to {SCALER_PATH}")

    # sequences
    train_seq = MangosteenSequence(train_paths, train_vols_scaled, BATCH_SIZE, IMG_SIZE, is_training=True)
    val_seq = MangosteenSequence(val_paths, val_vols_scaled, BATCH_SIZE, IMG_SIZE, is_training=False, shuffle=False)

    # build model
    base_model = Xception(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
    # if mixed precision, last Dense should output float32 for stable loss computation
    out = Dense(1, activation='linear', dtype='float32')(x) if ENABLE_MIXED_PRECISION else Dense(1, activation='linear')(x)
    model = Model(inputs=base_model.input, outputs=out)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE_INITIAL),
                  loss='huber', metrics=['mean_absolute_error'])

    # callbacks
    now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    checkpoint_path = os.path.join(CHECKPOINT_DIR, f"best_{now}.h5")
    callbacks = [
        ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1),
        EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=6, min_lr=1e-6, verbose=1),
        TensorBoard(log_dir=os.path.join("logs", now)),
        CSVLogger(os.path.join("logs", f"training_{now}.csv"))
    ]

    # train regression head
    logging.info("Training regression head...")
    history1 = model.fit(
        train_seq,
        validation_data=val_seq,
        epochs=INITIAL_EPOCHS,
        callbacks=callbacks,
        verbose=1
    )

    # fine-tune last block(s)
    # Xception block14 is named like 'block14...'; unlock some of the deeper layers
    for layer in base_model.layers:
        if 'block14' in layer.name or 'block13' in layer.name:
            layer.trainable = True

    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE_FINE_TUNE),
                  loss='huber', metrics=['mean_absolute_error'])

    # --- fine-tune last block(s) (แก้: เอา workers/use_multiprocessing ออก) ---
    logging.info("Fine-tuning last blocks of base model...")
    history2 = model.fit(
        train_seq,
        validation_data=val_seq,
        epochs=FINE_TUNE_EPOCHS,
        callbacks=callbacks,
        verbose=1
    )
    # save final model
    model.save(MODEL_SAVE_PATH)
    logging.info(f"✅ Model saved to {MODEL_SAVE_PATH}")
    logging.info(f"Best checkpoint: {checkpoint_path}")
    # ---- plot training history ----
    def plot_history(h1, h2, filename="training_plot.png"):
        df1 = pd.DataFrame(h1.history) if h1 else None
        df2 = pd.DataFrame(h2.history) if h2 else None
        plt.figure(figsize=(12, 5))
        # loss
        plt.subplot(1, 2, 1)
        if df1 is not None:
            plt.plot(df1['loss'], label='train_loss_phase1')
            plt.plot(df1['val_loss'], label='val_loss_phase1')
        if df2 is not None:
            plt.plot(df2['loss'], label='train_loss_phase2')
            plt.plot(df2['val_loss'], label='val_loss_phase2')
        plt.legend(); plt.title('Loss')
        # mae
        plt.subplot(1, 2, 2)
        if df1 is not None:
            plt.plot(df1['mean_absolute_error'], label='train_mae_phase1')
            plt.plot(df1['val_mean_absolute_error'], label='val_mae_phase1')
        if df2 is not None:
            plt.plot(df2['mean_absolute_error'], label='train_mae_phase2')
            plt.plot(df2['val_mean_absolute_error'], label='val_mae_phase2')
        plt.legend(); plt.title('MAE')
        plt.tight_layout()
        plt.savefig(filename)
        plt.close()
        logging.info(f"Training plot saved to {filename}")
    plot_history(history1, history2, filename=f"training_{now}.png")
    # ---- evaluate on validation in original units ----
    logging.info("Evaluating on validation set (original volume units)...")
    # predict all val (iterate val_seq)
    preds_scaled = []
    trues_scaled = []
    for Xb, yb in val_seq:
        p = model.predict(Xb, verbose=0)
        preds_scaled.append(p.reshape(-1))
        trues_scaled.append(yb.reshape(-1))
    preds_scaled = np.concatenate(preds_scaled, axis=0)
    trues_scaled = np.concatenate(trues_scaled, axis=0)
    # inverse transform
    preds_orig = scaler.inverse_transform(preds_scaled.reshape(-1, 1)).flatten()
    trues_orig = scaler.inverse_transform(trues_scaled.reshape(-1, 1)).flatten()
    mae_val = mean_absolute_error(trues_orig, preds_orig)
    rmse_val = math.sqrt(mean_squared_error(trues_orig, preds_orig))
    logging.info(f"Validation MAE (orig units): {mae_val:.4f}")
    logging.info(f"Validation RMSE (orig units): {rmse_val:.4f}")
    # save a small csv with true vs pred
    out_df = pd.DataFrame({"path": val_paths[:len(preds_orig)], "true": trues_orig, "pred": preds_orig})
    out_csv = f"val_predictions_{now}.csv"
    out_df.to_csv(out_csv, index=False)
    logging.info(f"Saved validation predictions to {out_csv}")
    
    

2025-09-13 21:08:13,142 INFO: Scaler saved to ./volume_scaler_finetune.save


Loading training data...


2025-09-13 21:08:13,989 INFO: Training regression head...


Epoch 1/100


d:\Coding\Python\Project\Mongoesteen\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


...

2025-09-13 21:48:11,315 INFO: Fine-tuning last blocks of base model...


Epoch 1/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.3109 - mean_absolute_error: 0.5359
Epoch 1: val_loss improved from 0.26587 to 0.26467, saving model to ./checkpoints\best_20250913-210813.h5


2025-09-13 21:48:40,201 WARNING: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


11/11 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - loss: 0.3190 - mean_absolute_error: 0.5481 - val_loss: 0.2647 - val_mean_absolute_error: 0.4762 - learning_rate: 1.0000e-05
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 0.2972 - mean_absolute_error: 0.5250
Epoch 2: val_loss did not improve from 0.26467
11/11 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - loss: 0.2916 - mean_absolute_error: 0.5112 - val_loss: 0.2728 - val_mean_absolute_error: 0.4933 - learning_rate: 1.0000e-05
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.2804 - mean_absolute_error: 0.5051
Epoch 3: val_loss did not improve from 0.26467
11/11 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - loss: 0.2767 - mean_absolute_error: 0.5025 - val_loss: 0.2914 - val_mean_absolute_error: 0.5217 - learning_rate: 1.0000e-05
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 0.2330 - mean_absolute_error: 0.4332
Epoch 4: val_loss did not improve from 0.26467
11/11 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - loss: 0.2404 - mean_absolute_error: 0.

2025-09-13 21:52:29,472 WARNING: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
2025-09-13 21:52:30,849 INFO: ✅ Model saved to ./mangosteen_volume_model_aug_finetune.h5
2025-09-13 21:52:30,850 INFO: Best checkpoint: ./checkpoints\best_20250913-210813.h5
2025-09-13 21:52:31,102 INFO: Training plot saved to training_20250913-210813.png
2025-09-13 21:52:31,104 INFO: Evaluating on validation set (original volume units)...
2025-09-13 21:52:35,298 INFO: Validation MAE (orig units): 12.1681
2025-09-13 21:52:35,298 INFO: Validation RMSE (orig units): 17.7695
2025-09-13 21:52:35,304 INFO: Saved validation predictions to val_predictions_20250913-210813.csv


In [3]:
# --- fine-tune last block(s) (แก้: เอา workers/use_multiprocessing ออก) ---
logging.info("Fine-tuning last blocks of base model...")
history2 = model.fit(
    train_seq,
    validation_data=val_seq,
    epochs=FINE_TUNE_EPOCHS,
    callbacks=callbacks,
    verbose=1
)
# save final model
model.save(MODEL_SAVE_PATH)
logging.info(f"✅ Model saved to {MODEL_SAVE_PATH}")
logging.info(f"Best checkpoint: {checkpoint_path}")
# ---- plot training history ----
def plot_history(h1, h2, filename="training_plot.png"):
    df1 = pd.DataFrame(h1.history) if h1 else None
    df2 = pd.DataFrame(h2.history) if h2 else None
    plt.figure(figsize=(12, 5))
    # loss
    plt.subplot(1, 2, 1)
    if df1 is not None:
        plt.plot(df1['loss'], label='train_loss_phase1')
        plt.plot(df1['val_loss'], label='val_loss_phase1')
    if df2 is not None:
        plt.plot(df2['loss'], label='train_loss_phase2')
        plt.plot(df2['val_loss'], label='val_loss_phase2')
    plt.legend(); plt.title('Loss')
    # mae
    plt.subplot(1, 2, 2)
    if df1 is not None:
        plt.plot(df1['mean_absolute_error'], label='train_mae_phase1')
        plt.plot(df1['val_mean_absolute_error'], label='val_mae_phase1')
    if df2 is not None:
        plt.plot(df2['mean_absolute_error'], label='train_mae_phase2')
        plt.plot(df2['val_mean_absolute_error'], label='val_mae_phase2')
    plt.legend(); plt.title('MAE')
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()
    logging.info(f"Training plot saved to {filename}")
plot_history(history1, history2, filename=f"training_{now}.png")
# ---- evaluate on validation in original units ----
logging.info("Evaluating on validation set (original volume units)...")
# predict all val (iterate val_seq)
preds_scaled = []
trues_scaled = []
for Xb, yb in val_seq:
    p = model.predict(Xb, verbose=0)
    preds_scaled.append(p.reshape(-1))
    trues_scaled.append(yb.reshape(-1))
preds_scaled = np.concatenate(preds_scaled, axis=0)
trues_scaled = np.concatenate(trues_scaled, axis=0)
# inverse transform
preds_orig = scaler.inverse_transform(preds_scaled.reshape(-1, 1)).flatten()
trues_orig = scaler.inverse_transform(trues_scaled.reshape(-1, 1)).flatten()
mae_val = mean_absolute_error(trues_orig, preds_orig)
rmse_val = math.sqrt(mean_squared_error(trues_orig, preds_orig))
logging.info(f"Validation MAE (orig units): {mae_val:.4f}")
logging.info(f"Validation RMSE (orig units): {rmse_val:.4f}")
# save a small csv with true vs pred
out_df = pd.DataFrame({"path": val_paths[:len(preds_orig)], "true": trues_orig, "pred": preds_orig})
out_csv = f"val_predictions_{now}.csv"
out_df.to_csv(out_csv, index=False)
logging.info(f"Saved validation predictions to {out_csv}")

2025-09-13 21:52:35,338 INFO: Fine-tuning last blocks of base model...


Epoch 1/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.3462 - mean_absolute_error: 0.5799
Epoch 1: val_loss improved from 0.26467 to 0.26382, saving model to ./checkpoints\best_20250913-210813.h5


2025-09-13 21:52:53,667 WARNING: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


11/11 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - loss: 0.3161 - mean_absolute_error: 0.5441 - val_loss: 0.2638 - val_mean_absolute_error: 0.4774 - learning_rate: 1.0000e-06
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.2996 - mean_absolute_error: 0.5262
Epoch 2: val_loss did not improve from 0.26382
11/11 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - loss: 0.2999 - mean_absolute_error: 0.5268 - val_loss: 0.2645 - val_mean_absolute_error: 0.4792 - learning_rate: 1.0000e-06
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.2823 - mean_absolute_error: 0.5033
Epoch 3: val_loss did not improve from 0.26382
11/11 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - loss: 0.3059 - mean_absolute_error: 0.5296 - val_loss: 0.2658 - val_mean_absolute_error: 0.4799 - learning_rate: 1.0000e-06
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.2946 - mean_absolute_error: 0.5178
Epoch 4: val_loss did not improve from 0.26382
11/11 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - loss: 0.2994 - mean_absolute_error: 0.

2025-09-13 21:56:39,556 WARNING: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
2025-09-13 21:56:39,909 INFO: ✅ Model saved to ./mangosteen_volume_model_aug_finetune.h5
2025-09-13 21:56:39,911 INFO: Best checkpoint: ./checkpoints\best_20250913-210813.h5
2025-09-13 21:56:40,106 INFO: Training plot saved to training_20250913-210813.png
2025-09-13 21:56:40,106 INFO: Evaluating on validation set (original volume units)...
2025-09-13 21:56:43,383 INFO: Validation MAE (orig units): 12.2008
2025-09-13 21:56:43,384 INFO: Validation RMSE (orig units): 17.6673
2025-09-13 21:56:43,386 INFO: Saved validation predictions to val_predictions_20250913-210813.csv
